#### **BigBasket Product Query Engine  Notebook 2**
**It contains**
1. importing vector embedding from file space.pkl which is python dic object vector DB 
2. storing vector in vector DB Qdrant 
3. saving qdrant for api
4. saving bert model for api
5. saving sentence transformer for api

In [ ]:
!pip install -q qdrant-client
!pip install -q sentence-transformers
!pip install -q torch


In [ ]:
import warnings
warnings.filterwarnings('ignore')
from transformers import pipeline
from qdrant_client import QdrantClient
from qdrant_client.http import models
from tqdm.auto import tqdm
from sentence_transformers import SentenceTransformer
import torch
import pickle
import numpy as np

In [ ]:
VECTOR_SPACE_PATH = "/kaggle/input/chaabi/space.pkl"
with open(VECTOR_SPACE_PATH, 'rb') as file:
    vs = pickle.load(file)

In [ ]:
client = QdrantClient(":memory:")

In [ ]:
collection_name = "qdrant-space"

collections = client.get_collections()
if collection_name not in [c.name for c in collections.collections]:
    client.recreate_collection(
        collection_name=collection_name,
        vectors_config=models.VectorParams(
            size=384,
            distance=models.Distance.COSINE,
        ),
    )
print(client.get_collections())

In [ ]:
%%time

 
total_records = len(vs["payload"]) # total records data
_payload = vs["payload"]
_emb = vs["emb"]
ids = list(range(0,total_records))

batch_size = 2  # specify batch size according to your RAM and compute, higher batch size = more RAM usage

client.upsert(
    collection_name=collection_name,
    points=models.Batch(ids=ids, vectors=_emb, payloads=_payload),
)


collection_vector_count = client.get_collection(collection_name=collection_name).vectors_count
print(f"Vector count in collection: {collection_vector_count}")


In [ ]:
# dump qdrant space note keep sentence transformer in environment 

with open('qdrant_space_client.pkl', 'wb') as file:
    pickle.dump(client, file)

In [ ]:
if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"
    
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    retriever = SentenceTransformer("multi-qa-MiniLM-L6-cos-v1", device=device)
# retriever = SentenceTransformer("multi-qa-MiniLM-L6-cos-v1", device=device)
retriever



In [ ]:
# dump encoder
import pickle

# Deserialize the data from the pickle file
with open('encodermodel.pkl', 'wb') as file:
    pickle.dump(retriever, file)

In [ ]:
model_name = "bert-large-uncased-whole-word-masking-finetuned-squad"

# load the reader model into a question-answering pipeline
reader = pipeline("question-answering", model=model_name, tokenizer=model_name)


In [ ]:
# dump bert model 
# reader.save_pretrained("bert-question-answering")
with open('bert-question-answering.pkl', 'wb') as file:
    pickle.dump(reader, file)